In [ ]:
import pandas as pd
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import matplotlib.pyplot as plt
# %matplotlib notebook
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

os.environ['KAGGLE_CONFIG_DIR'] = r'C:\Users\amitb\.kaggle'
api = KaggleApi()
api.authenticate()
print("Kaggle authentication successful!")

In [ ]:
data = pd.read_csv(r"C:\code_projects\applied_DS\rain_in_australia_classifier\weatherAUS.csv")
print(f'There are {data.shape[0]} samples within the dataset')
data.head(10)

In [ ]:
data.describe(include='all')

In [ ]:
numeric_cols = data.select_dtypes(include=["number"]).columns.tolist()

for numeric_col in numeric_cols:
    data[numeric_col] = data[numeric_col].astype(float)


categorical_cols = data.select_dtypes(include=["object"]).columns.tolist()
categorical_cols.remove('Date')

for categorical_col in categorical_cols:
    data[categorical_col] = data[categorical_col].astype('category')

data['Date'] = data['Date'].astype('datetime64[ns]')

data.dtypes

# Handling missing values


In [ ]:
# checking NA

mising_values_col_sum = pd.Series(data.isnull().sum(), name='% of NA').apply(lambda x: f"{(x / data.shape[0] * 100):.2f}").astype('float').sort_values(ascending=False)
mising_values_col_sum

In [ ]:
features = ['Sunshine', 'Evaporation', 'Cloud3pm','Cloud9am','Pressure9am','Pressure3pm']

n_features = len(features)
fig, axes = plt.subplots(n_features, 1, figsize=(8, 4*n_features))  # vertical layout

for i, feature in enumerate(features):
    axes[i].hist(data[feature], bins=20, color='skyblue', edgecolor='black')
    axes[i].set_title(f'{feature} | % missing values {mising_values_col_sum[feature]}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# detect features with/without normal distribution


results = {'feature': [], 'Skewness': [], 'Excess_Kurtosis': []}

for column in features:
    x = data[column].dropna()
    results['feature'].append(column)
    results['Skewness'].append(skew(x))
    results['Excess_Kurtosis'].append(kurtosis(x))

pd.DataFrame(results)



In [ ]:
# use median to imput features with normal distribution

results = pd.DataFrame(results)

features_to_imput_with_median = [
    f for f, skew, kurt in zip(results.feature, results.Skewness, results.Excess_Kurtosis)
    if abs(skew) < 0.1 and abs(kurt) < 0.5
]

for feature in features_to_imput_with_median:
    feature_median = data[feature].median()
    data[feature] = data[feature].fillna(feature_median)

In [ ]:
#fillna with random choise to preserves the skewed distribution and extreme values (for Evaporation which dosen't have bi-mode / normal dist)

observed = data['Evaporation'].dropna()
data['Evaporation'] = data['Evaporation'].fillna(np.random.choice(observed))

plt.figure(figsize=(8, 5))
plt.hist(data['Evaporation'].dropna(), bins=30, color='skyblue', edgecolor='black')
plt.title('Histogram of Evaporation')
plt.xlabel('Evaporation')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
# Predict cluster for missing values randomly based on cluster weights for features with bi-modal distribution

bi_mode_features = ['Cloud3pm', 'Cloud9am', 'Sunshine']

for col in bi_mode_features:
    observed = data[col].dropna().values.reshape(-1,1)

    # Fit 2-component GMM
    gmm = GaussianMixture(n_components=2, random_state=42)
    gmm.fit(observed)

    # Missing indices
    missing_idx = data[col].isna()
    n_missing = missing_idx.sum()

    # Assign clusters to missing values
    sampled_cluster = np.random.choice(2, size=n_missing, p=gmm.weights_)

    # Get cluster labels for observed values
    cluster_labels = gmm.predict(observed)

    # Precompute cluster-wise observed values for faster sampling
    cluster_values = [observed[cluster_labels == k].flatten() for k in range(2)]

    # Vectorized imputation
    imputed_values = np.array([np.random.choice(cluster_values[k]) for k in sampled_cluster])

    data.loc[missing_idx, col] = imputed_values


missing_after_GaussianMixture = pd.Series(data[bi_mode_features].isna().sum(),name='% missing')
missing_after_GaussianMixture

n_features = len(bi_mode_features)

fig, axes = plt.subplots(n_features, 1, figsize=(8, 4*n_features))  # vertical layout

for i, feature in enumerate(bi_mode_features):
    axes[i].hist(data[feature], bins=20, color='skyblue', edgecolor='black')
    axes[i].set_title(f'{feature} | % missing values: {missing_after_GaussianMixture[feature]}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
#final review of NA values info (for these features with less than 10% missing values

cols_na = pd.Series(data.isnull().sum(), name='% of NA').apply(lambda x: f"{(x / data.shape[0] * 100):.2f}").astype('float').sort_values(ascending=False)

filtered = cols_na[cols_na > 0]

features = filtered.index.tolist()

results = {'feature': [], 'Skewness': [], 'Excess_Kurtosis': []}

for column in features:
    if column in numeric_cols:
        x = data[column].dropna()
        results['feature'].append(column)
        results['Skewness'].append(skew(x))
        results['Excess_Kurtosis'].append(kurtosis(x))

data_results = pd.DataFrame(results)

In [ ]:
data_results

In [ ]:

# features_numeric = [x for x in filtered.index.tolist() if x in numeric_cols]
#
# n_features = len(features_numeric)
# fig, axes = plt.subplots(n_features, 1, figsize=(8, 4*n_features))  # vertical layout
#
# for i, feature in enumerate(features_numeric):
#     axes[i].hist(data[feature], bins=20, color='skyblue', edgecolor='black')
#     axes[i].set_title(f'{feature}')
#     axes[i].set_xlabel(f'{feature}')
#     axes[i].set_ylabel('Frequency')
#
# plt.tight_layout()
# plt.show()

In [ ]:
normal_features = data_results[
    (data_results['Skewness'].abs() < 0.1) & (data_results['Excess_Kurtosis'].abs() < 0.5)
]['feature'].tolist()

skewed_features = data_results[
    ~data_results['feature'].isin(normal_features)
]['feature'].tolist()

# Impute normal-ish features with mean
for feature in normal_features:
    data[feature] = data[feature].fillna(data[feature].mean())

# Impute highly skewed features with median
for feature in skewed_features:
    data[feature] = data[feature].fillna(data[feature].median())

In [ ]:
nan_results = pd.Series(data.isnull().sum(), name='% of NA').apply(lambda x: f"{(x / data.shape[0] * 100):.2f}").astype('float').sort_values(ascending=False)
cols_to_imput = nan_results[nan_results.values >0].index
data[cols_to_imput].info()

In [ ]:
data[cols_to_imput].describe(include='all')


In [ ]:
wind_cols = [x for x in cols_to_imput if 'Wind' in str(x)]

common_values = set(data[wind_cols[0]].unique())
for col in wind_cols[1:]:
    common_values &= set(data[col].unique())

print("Common values across all columns:", common_values)

In [ ]:
for col in wind_cols:
    data[col]= data[col].fillna(data[col].mode()[0])

# Adding Features

In [ ]:
#create categorical Season feature

def get_season(date):
    day = date.day
    month = date.month

    if (month == 12 and day >= 21) or (month <= 3 and (month < 3 or day <= 21)):
        return 'Summer'
    elif (month == 3 and day >= 22) or (month <= 6 and (month < 6 or day <= 21)):
        return 'Autumn'
    elif (month == 6 and day >= 22) or (month <= 9 and (month < 9 or day <= 21)):
        return 'Winter'
    elif (month == 9 and day >= 22) or (month <= 12 and (month < 12 or day <= 20)):
        return 'Spring'

data['Season'] = data['Date'].map(get_season)
data['Season'] = data['Season'].astype('category')

data = pd.get_dummies(data, columns=["Season"], drop_first=False)


In [ ]:
binary_dict = {'No': 0, 'Yes':1}

data['RainToday']=data['RainToday'].map(binary_dict)
data['RainTomorrow']=data['RainTomorrow'].map(binary_dict)

In [ ]:
#extract COSIN for day, month and year

data["day_of_week"] = data["Date"].dt.weekday      # 0 = Monday
data["month"] = data["Date"].dt.month
data["day_of_year"] = data["Date"].dt.dayofyear
data["year"] = data["Date"].dt.year

# Cyclical encoding
data["day_of_week_sin"] = np.sin(2 * np.pi * data["day_of_week"] / 7)
data["day_of_week_cos"] = np.cos(2 * np.pi * data["day_of_week"] / 7)

data["month_sin"] = np.sin(2 * np.pi * data["month"] / 12)
data["month_cos"] = np.cos(2 * np.pi * data["month"] / 12)

data["day_of_year_sin"] = np.sin(2 * np.pi * data["day_of_year"] / 365)
data["day_of_year_cos"] = np.cos(2 * np.pi * data["day_of_year"] / 365)

data.drop(columns='Date',inplace=True)

In [ ]:
data.dropna(subset='RainTomorrow',inplace=True,axis=0)
data['RainTomorrow'].isna().sum()

In [ ]:
data = pd.get_dummies(data, columns=["RainToday"], drop_first=False)

In [ ]:
#Label encoing for categorical columns

Location = LabelEncoder()
data['Location'] = Location.fit_transform(data['Location'])

wind_categories = sorted(data[wind_cols[0]].unique())  # replace 'wind1' with one of the 3
le_wind = LabelEncoder()
le_wind.fit(wind_categories)

for col in wind_cols:
    data[col + '_encoded'] = le_wind.transform(data[col])

data.drop(columns=[x for x in data.columns if 'Wind' in x and 'encoded' not in x], inplace=True)

In [ ]:
data.head()

# spliting

In [ ]:
x = data.drop(columns='RainTomorrow')
y = data['RainTomorrow']

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state = 42)


In [ ]:
#detect columns for standard scaler

numeric_cols = data.select_dtypes(include=[np.number]).columns
print(numeric_cols)

X_train = X_train[['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm']]

X_test = X_test[['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm']]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply same scaler to test data
X_test_scaled = scaler.transform(X_test)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from xgboost import XGBClassifier

def train_xgb_with_test_metrics(X_train, X_test, y_train, y_test, param_grid=None, n_splits=5, random_state=42):
    """
    Train XGBClassifier with Stratified K-Fold grid search.
    Evaluate metrics only on held-out test set.
    Plots confusion matrix, ROC-AUC, and feature importance.

    Parameters:
    -----------
    X_train, X_test : pd.DataFrame or np.array
        Feature matrices for train/test.
    y_train, y_test : pd.Series or np.array
        Target arrays for train/test.
    param_grid : dict
        Hyperparameter grid for GridSearchCV.
    n_splits : int
        Number of folds for StratifiedKFold.
    random_state : int
        Random seed.

    Returns:
    --------
    best_model : fitted GridSearchCV best estimator
    """

    # Default hyperparameter grid
    if param_grid is None:
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1],
            'colsample_bytree': [0.8, 1],
            'gamma': [0, 1]
        }

    # Stratified K-Fold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # XGBClassifier
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=random_state)

    # Grid search
    grid = GridSearchCV(
        estimator=xgb,
        param_grid=param_grid,
        scoring='f1',
        n_jobs=-1,
        cv=skf,
        verbose=1
    )

    # Fit on training set
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    print("Best hyperparameters:", grid.best_params_)

    # Predict on test set
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:,1]

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix (Test Set)")
    plt.show()

    # Classification Report
    print("Classification Report (Test Set):\n")
    print(classification_report(y_test, y_pred))

    # ROC-AUC Curve
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    auc_score = roc_auc_score(y_test, y_proba)
    plt.figure(figsize=(6,5))
    plt.plot(fpr, tpr, label=f'ROC-AUC = {auc_score:.3f}')
    plt.plot([0,1],[0,1],'--', color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve (Test Set)")
    plt.legend()
    plt.show()

    # Feature Importance
    importance = best_model.feature_importances_
    feature_names = X_train.columns if isinstance(X_train, pd.DataFrame) else [f"f{i}" for i in range(X_train.shape[1])]
    feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
    feat_df = feat_df.sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(8,6))
    sns.barplot(x='Importance', y='Feature', data=feat_df)
    plt.title("Feature Importance")
    plt.show()

    return best_model, grid


In [ ]:
best_model, grid = train_xgb_with_test_metrics(X_train_scaled, X_test_scaled, y_train, y_test)